In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Cargar los datos desde el archivo CSV
archivo_csv = 'live.csv'
data = pd.read_csv(archivo_csv)

# Seleccionar las columnas 'num_reactions' y 'num_shares'
columnas_seleccionadas = ['num_reactions', 'num_shares']
X = data[columnas_seleccionadas].values

# Definir la función para calcular la distancia euclidiana entre dos puntos
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

# Definir la función para encontrar los vecinos de un punto dentro de un radio epsilon
def find_neighbors(X, point_idx, epsilon):
    neighbors = []
    for i, x in enumerate(X):
        if euclidean_distance(X[point_idx], x) <= epsilon:
            neighbors.append(i)
    return neighbors

# Implementar el algoritmo DBSCAN
def dbscan(X, epsilon, min_samples):
    labels = np.zeros(len(X), dtype=int)  # Etiquetas iniciales, todos puntos como ruido (0)
    cluster_id = 1  # Identificador del clúster
    for i, x in enumerate(X):
        if labels[i] != 0:  # Si ya está etiquetado, continuar
            continue
        neighbors = find_neighbors(X, i, epsilon)
        if len(neighbors) < min_samples:
            labels[i] = -1  # Marcar como ruido
        else:
            labels[i] = cluster_id
            expand_cluster(X, labels, i, neighbors, cluster_id, epsilon, min_samples)
            cluster_id += 1
    return labels

# Función para expandir un clúster
def expand_cluster(X, labels, point_idx, neighbors, cluster_id, epsilon, min_samples):
    for neighbor_idx in neighbors:
        if labels[neighbor_idx] == 0:  # Si no está etiquetado
            labels[neighbor_idx] = cluster_id
            new_neighbors = find_neighbors(X, neighbor_idx, epsilon)
            if len(new_neighbors) >= min_samples:
                neighbors.extend(new_neighbors)

# Parámetros de DBSCAN
epsilon = 0.5  # Radio epsilon
min_samples = 5  # Mínimo de puntos en un clúster

# Ejecutar DBSCAN
dbscan_labels = dbscan(X, epsilon, min_samples)

# Agregar las etiquetas de DBSCAN al DataFrame original
data['DBSCAN_Labels'] = dbscan_labels

# Visualizar los resultados
plt.scatter(data['num_reactions'], data['num_shares'], c=dbscan_labels, cmap='viridis')
plt.xlabel('Número de Reacciones')
plt.ylabel('Número de Compartidos')
plt.title('DBSCAN Clustering (sin sklearn)')
plt.show()


KeyboardInterrupt: 